In [1]:
import sys
!{sys.executable} -m pip install nltk

In [2]:
!{sys.executable} -m pip install textblob

In [1]:
#Prevent Large File crashing on Jupyter Notebook
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob, Word
from matplotlib.font_manager import FontProperties
import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
import datetime
import warnings 
#sns.set_style("darkgrid",{"axes.axisbelow" : False })
warnings.simplefilter('ignore')
import string
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
import string   
import re
from nltk.stem import WordNetLemmatizer

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
STOPWORDS = stopwords.words("english") #stopwords are the most common unnecessary words. eg is, he, that, etc.

[nltk_data] Downloading package stopwords to C:\Users\Tien
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Tien
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package punkt to C:\Users\Tien
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tien Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [5]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii') # A function to remove emojis from the reviews

In [6]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [7]:
def clean_text(text):
    #ps=PorterStemmer()
    #wordnet_lemmatizer = WordNetLemmatizer()
    if not isinstance(text, str):
        return ''
    text=deEmojify(text) # remove emojis
    text_cleaned="".join([x for x in text if x not in string.punctuation]) # remove punctuation
    
    text_cleaned=re.sub(' +', ' ', text_cleaned) # remove extra white spaces
    text_cleaned=text_cleaned.lower() # converting to lowercase
    tokens=text_cleaned.split(" ")
    tokens=[token for token in tokens if token not in STOPWORDS] # Taking only those words which are not stopwords
    
    #Converting to lemma
    #text_cleaned=" ".join([wordnet_lemmatizer.lemmatize(token) for token in tokens])
    #text_cleaned=" ".join([ps.stem(token) for token in tokens])
    text_cleaned = lemmatize_with_postag(str(tokens))
    for r in (("\' ", ''), ('\'', ''), ('[',''),  (']','')):
        text_cleaned = text_cleaned.replace(*r)
    #print(text_cleaned)                            
    return text_cleaned

# ---------------------------------------------------
# Cleaning Amazon text reviews
 - remove emojis
 - remove punctuation
 - remove remove extra white spaces
 - converting to lowercase 
 - remove stopwords
 - using lemma instead of stem   
  _lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, often leading to incorrect meanings and spelling errors._

In [ ]:
### l = []
with open('amazon_official.tsv', 'r') as f:
    next(f)
    with open('amazon_cleaned_official.tsv', 'w') as g:
        g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+'\t'+'reviewText' + '\t' + 'cleaned_text' + '\n')
        for i, line in enumerate(f):
            #if i == 3:
                #break
            line = line.strip().split('\t')
            overall = line[0]
            reviewTime = line[1]
            asin = line[2]
            if len(line) > 3:
                reviewText = line[3] 
                cleaned_text = clean_text(reviewText)
                g.write(overall + '\t' + reviewTime + '\t' + asin +'\t' + reviewText + '\t' + cleaned_text + '\n')
            else:
                g.write(overall + '\t' + reviewTime + '\t' + asin +'\t' + '' + '\t' + '' + '\n')

In [11]:
raw_data = pd.read_csv('amazon_cleaned_official.tsv', sep = '\t', nrows=3)

In [14]:
pd.options.display.max_colwidth = 200
raw_data

,overall,reviewTime,asin,reviewText,cleaned_text
0,5.0,08 12 2005,1713353,This book is a winner with both of my boys. They really enjoy the pictures and the story. It's a classic.,book winner boy really enjoy picture story classic
1,5.0,03 30 2005,1713353,"The King, the Mice and the Cheese by Nancy Gurney is an excellent children's book. It is one that I well remember from my own childhood and purchased for my daughter who loves it. It is about a ...",king mouse cheese nancy gurney excellent childrens book one well remember childhood purchased daughter love king trouble rude mouse eating cheese consults wise men suggest cat chase away mouse cat...
2,5.0,04 4 2004,1713353,"My daughter got her first copy from her great-grandmother (it had been my father's when he was a child). She loves it so much that she has worn out two copies now. Colorful pictures, easy to follo...",daughter got first copy greatgrandmother father child love much worn two copy colorful picture easy follow word wonderful moral sharing thinking problem easy way isnt always best way


# ---------------------------------------------------------------------